<a href="https://colab.research.google.com/github/techwithanirudh/Automatic-Image-Captioning/blob/master/FullWebsiteGrammerCorrecter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install language-check
!pip install pycontractions
!pip install flask_ngrok
!pip install flask-cors

In [ ]:
import language_check 

# ep 8

tool = language_check.LanguageTool('en-US') 
line = 'Hi I an umbrella'
matches = tool.check(line) 
for mistake in matches: 
  mistake = str(mistake)
  mistake = mistake.split('Message: ')[1].replace('^', '')
  # mistake = mistake.split('})]')[1]
  print(mistake) 

In [2]:
import os

try:
  os.mkdir('/content/templates')
  os.mkdir('/content/static/')
  os.mkdir('/content/static/css')
  os.mkdir('/content/static/js')
except FileExistsError:
  pass

open('/content/templates/index.html', 'a')
with open('/content/templates/index.html', 'w') as htmlF:
  htmlF.write('''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>English Grammer</title>
    <!-- <link rel="icon" type="image/x-icon" href="https://media.geeksforgeeks.org/wp-content/cdn-uploads/gfg_200X200.png">  -->
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <script>
        function compute() {
            var msgElement = document.getElementById('msg');
            if (msgElement != null)
            {
                var msg = msgElement.value;
                msgElement.value = '';
            }
            var id = {{ngrok_add}};
            var request = new XMLHttpRequest();
                request.open('GET', 'https://' + id + '.ngrok.io/api/req/' + msg, true);
                request.onload = function() {
                    var res = this.response.toString();
                    if (request.status >= 200 && request.status < 400) {
                        console.log(res);
                        show(res);
                    }
                    else {
                        console.log('ERROR');
                    }
                }
                request.send();
            }

        function clickPress(event) {
            if (event.keyCode == 13) {
                var msgElement = document.getElementById('msg');
                compute();
            }
        }

        function show(txt) {
            var showElement = document.getElementById('res');
            showElement.innerHTML = '<pre>' + txt + '</pre>';
        }
    </script>
    <div class="title">
    <h1>English Grammer
        <svg xmlns="https://www.w3.org/2000/svg" width="30" height="30" fill="currentColor" class="bi bi-book" viewBox="0 0 16 16">
            <path d="M1 2.828c.885-.37 2.154-.769 3.388-.893 1.33-.134 2.458.063 3.112.752v9.746c-.935-.53-2.12-.603-3.213-.493-1.18.12-2.37.461-3.287.811V2.828zm7.5-.141c.654-.689 1.782-.886 3.112-.752 1.234.124 2.503.523 3.388.893v9.923c-.918-.35-2.107-.692-3.287-.81-1.094-.111-2.278-.039-3.213.492V2.687zM8 1.783C7.015.936 5.587.81 4.287.94c-1.514.153-3.042.672-3.994 1.105A.5.5 0 0 0 0 2.5v11a.5.5 0 0 0 .707.455c.882-.4 2.303-.881 3.68-1.02 1.409-.142 2.59.087 3.223.877a.5.5 0 0 0 .78 0c.633-.79 1.814-1.019 3.222-.877 1.378.139 2.8.62 3.681 1.02A.5.5 0 0 0 16 13.5v-11a.5.5 0 0 0-.293-.455c-.952-.433-2.48-.952-3.994-1.105C10.413.809 8.985.936 8 1.783z"/>
        </svg>
    </h1>
    </div>
    <div class='chat-footer'>
        <textarea placeholder='Type a sentence...' id='msg' onkeypress="clickPress(event)"></textarea>
    </div>
    <p id='res'></p>
    <button class="btnCompute" onclick="compute()">Compute</button>
</body>
</html>
  ''')
  htmlF.close()


open('/content/static/css/styles.css', 'a')
with open('/content/static/css/styles.css', 'w') as cssF:
  cssF.write('''.btnCompute {
    background-color: #008CBA; /* Blue */
    border: none;
    color: white;
    padding: 15px 32px;
    text-align: center;
    text-decoration: none;
    display: inline-block;
    font-size: 16px;
    margin: 4px 2px;
    cursor: pointer;
  }
  *{
	box-sizing: border-box;
	font-family: sans-serif;
}

.chat-footer {
	display: flex;
	justify-content: center;
	align-items: center;
	border-radius: 60px;
	position: relative;
	cursor: pointer;
}
.chat-footer textarea {
	display: block;
	flex: 1;
	width: 100%;
	height: 50px;
	border-radius: 60px;
	margin: 5px;
	padding: 10px;
	outline: none;
	font-size: 19px;
	padding-left: 30px;
	padding-right: 30px;
	border: 2px solid #ccc;
	color: #555;
	resize: none;
}
.chat-footer .mic {
	display: block;
	width: 55px;
	height: 55px;
	margin-right: 20px;
}
  ''')
  cssF.close()

In [ ]:
import language_check 
from flask import Flask, render_template
# from flask_ngrok import run_with_ngrok
# import flask_ngrok
import requests
import threading, time
from flask_cors import CORS
import atexit
import json
import os
import platform
import shutil
import subprocess
import tempfile
import time
import zipfile
from pathlib import Path
from threading import Timer

import requests

ngrok_address = ''

def _run_ngrok():
    ngrok_path = str(Path(tempfile.gettempdir(), "ngrok"))
    _download_ngrok(ngrok_path)
    system = platform.system()
    if system == "Darwin":
        command = "ngrok"
    elif system == "Windows":
        command = "ngrok.exe"
    elif system == "Linux":
        command = "ngrok"
    else:
        raise Exception(f"{system} is not supported")
    executable = str(Path(ngrok_path, command))
    os.chmod(executable, 777)

    ngrok = subprocess.Popen([executable, 'http', '5000'])
    atexit.register(ngrok.terminate)
    localhost_url = "http://localhost:4040/api/tunnels"  # Url with tunnel details
    time.sleep(1)
    tunnel_url = requests.get(localhost_url).text  # Get the tunnel information
    j = json.loads(tunnel_url)

    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
    tunnel_url = tunnel_url.replace("https", "http")
    return tunnel_url


def _download_ngrok(ngrok_path):
    if Path(ngrok_path).exists():
        return
    system = platform.system()
    if system == "Darwin":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-darwin-amd64.zip"
    elif system == "Windows":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-amd64.zip"
    elif system == "Linux":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"
    else:
        raise Exception(f"{system} is not supported")
    download_path = _download_file(url)
    with zipfile.ZipFile(download_path, "r") as zip_ref:
        zip_ref.extractall(ngrok_path)


def _download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    download_path = str(Path(tempfile.gettempdir(), local_filename))
    with open(download_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
    return download_path

def start_ngrok():
    global ngrok_address
    ngrok_address = _run_ngrok()
    print(f" * Running on {ngrok_address}")
    print(f" * Traffic stats available on http://127.0.0.1:4040")


def run_with_ngrok(app):
    """
    The provided Flask app will be securely exposed to the public internet via ngrok when run,
    and the its ngrok address will be printed to stdout
    :param app: a Flask application object
    :return: None
    """
    old_run = app.run

    def new_run():
        thread = Timer(1, start_ngrok)
        thread.setDaemon(True)
        thread.start()
        old_run()
    app.run = new_run

####################

app = Flask(__name__)
print('ng', run_with_ngrok(app), 'ng')
CORS(app)
tool = language_check.LanguageTool('en-US') 

@app.route('/api/req/<sentence>')
def apiReq(sentence):
  sentence = sentence.replace('-', ' ')
  matches = tool.check(sentence) 
  mistakes = ''
  print(sentence)
  for mistake in matches: 
    mistake = str(mistake)
    mistake = mistake.split('Message: ')[1]
    print(mistake)
    if '^' in mistake:
      mistakes += ''
    mistakes += mistake
  return mistakes

@app.route('/')
def index():
  print(ngrok_address)
  Var_ngrok_add = ngrok_address.split('http://')[1]
  Var_ngrok_add = Var_ngrok_add.split('.ngrok.io')[0]
  print(Var_ngrok_add)
  return render_template('index.html', ngrok_add=Var_ngrok_add)

# def ngrok_url():
#   global ngrok_address
#   while True:
#     try:
#       # print(ngrok_address)
#       pass
#     except Exception as e:
#       print(e)

if __name__ == '__main__':
  # thread = Timer(1, ngrok_url)
  # thread.setDaemon(True)
  # thread.start()
  app.run()
  # threading.Thread(target=app_run()).start()


ng None ng
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1c9a1a8601da.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Jan/2021 18:18:04] "GET / HTTP/1.1" 200 -


http://1c9a1a8601da.ngrok.io
1c9a1a8601da


127.0.0.1 - - [15/Jan/2021 18:18:04] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2021 18:18:06] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2021 18:18:12] "GET /styles.css HTTP/1.1" 404 -
127.0.0.1 - - [15/Jan/2021 18:18:13] "GET /api/req/I%20am%20frsdfs HTTP/1.1" 200 -


I am frsdfs
Possible spelling mistake found
I am frsdfs
     ^^^^^^


In [ ]:
# RUN THIS ON COLAB

'''# !sudo apt install openjdk-8-jdk
# !sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
# !pip install language-check
# !pip install pycontractions
!pip install flask_ngrok

import language_check 
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)
tool = language_check.LanguageTool('en-US') 

@app.route('/api/req/<sentence>')
def apiReq(sentence):
  sentence = sentence.replace('-', ' ')
  matches = tool.check(sentence) 
  mistakes = ''
  print(sentence)
  for mistake in matches: 
    mistake = str(mistake)
    mistake = mistake.split('Message: ')[1]
    print(mistake)
    if '^' in mistake:
      mistakes += '\n'  
    mistakes += mistake
  return mistakes

if __name__ == '__main__':
  app.run()'''

from PyQt5 import QtCore, QtWidgets
import requests

class Ui_MainWindow(object):
    def btnCheck_clicked(self):
        self.text = self.txtSentence.toPlainText().replace(' ', '-')
        # TODO-Fake: The id should be changed by seeing colab You should copy id from url and paste
        self.id = '3e986e04abe1'
        if self.text.replace('\n', '').replace('-', '') != '':
            res = requests.get(f'https://{self.id}.ngrok.io/api/req/{self.text}').text
            print(res)
            if res != '':
                self.lblShowgrammatical_rules.setStyleSheet('background-color: red')
            else:
                res = self.text
                self.lblShowgrammatical_rules.setStyleSheet('background-color: green')

            self.lblShowgrammatical_rules.setText(_translate('MainWindow', res))
            self.lblShowSuggestion.setText(_translate('MainWindow', res))

    def setupUi(self, MainWindow):
        MainWindow.setObjectName('MainWindow')
        MainWindow.resize(800, 730)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName('centralwidget')
        self.lblHelpText = QtWidgets.QLabel(self.centralwidget)
        self.lblHelpText.setGeometry(QtCore.QRect(6, 10, 141, 21))
        self.lblHelpText.setObjectName('lblHelpText')
        self.lblSentence = QtWidgets.QLabel(self.centralwidget)
        self.lblSentence.setGeometry(QtCore.QRect(30, 60, 161, 21))
        self.lblSentence.setObjectName('lblSentence')
        self.txtSentence = QtWidgets.QTextEdit(self.centralwidget)
        self.txtSentence.setGeometry(QtCore.QRect(70, 100, 431, 61))
        self.txtSentence.setObjectName('txtSentence')
        self.btnCheck = QtWidgets.QPushButton(self.centralwidget)
        self.btnCheck.setGeometry(QtCore.QRect(240, 180, 81, 31))
        self.btnCheck.setObjectName('btnCheck')
        self.lblSuggestion = QtWidgets.QLabel(self.centralwidget)
        self.lblSuggestion.setGeometry(QtCore.QRect(40, 250, 61, 16))
        self.lblSuggestion.setObjectName('lblSuggestion')
        self.lblShowSuggestion = QtWidgets.QLabel(self.centralwidget)
        self.lblShowSuggestion.setGeometry(QtCore.QRect(10, 270, 781, 141))
        self.lblShowSuggestion.setObjectName('lblShowSuggestion')
        self.lblgrammatical_rules = QtWidgets.QLabel(self.centralwidget)
        self.lblgrammatical_rules.setGeometry(QtCore.QRect(40, 460, 181, 16))
        self.lblgrammatical_rules.setObjectName('lblgrammatical_rules')
        self.lblShowgrammatical_rules = QtWidgets.QLabel('Light green', self.centralwidget)
        self.lblShowgrammatical_rules.setGeometry(QtCore.QRect(10, 480, 781, 161))
        self.lblShowgrammatical_rules.setObjectName('lblShowgrammatical_rules')
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 800, 21))
        self.menubar.setObjectName('menubar')
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName('statusbar')
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        global _translate
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate('MainWindow', 'MainWindow'))
        self.lblHelpText.setText(_translate('MainWindow', '?<Put Your Help Text Here>'))
        self.lblSentence.setText(_translate('MainWindow', 'Please type your sentences here:'))
        self.btnCheck.setText(_translate('MainWindow', 'Check'))
        self.btnCheck.clicked.connect(self.btnCheck_clicked)
        self.lblSuggestion.setText(_translate('MainWindow', 'Suggestion:'))
        self.lblShowSuggestion.setText(_translate('MainWindow', 'Label'))
        self.lblgrammatical_rules.setText(_translate('MainWindow', 'The grammatical rules needed to use:'))
        self.lblShowgrammatical_rules.setText(_translate('MainWindow', 'Label'))

if __name__ == '__main__':
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())
